<a href="https://colab.research.google.com/github/fervo03/DFC_2020/blob/master/DFC609_Project_20200613.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from io import StringIO

path = '/content/drive/My Drive/Colab Notebooks/20200511_055550.txt'
fhand = open(path, 'rt')
trimed_data = 'Type Time C_price C_quantity C_direction Ask_price Ask_quantity Bid_price Bid_quantity Move_to*\n'

pd.set_option('display.max_row', 500)

In [4]:
pre_ask_p = 0

for line in fhand:
  time = line[-11:-2]
  data_type = line[:2]

  try:
    itime = int(time)
  except:
    continue

  if itime < 90000000 or itime > 152000000 :
    continue
  
  if data_type == 'A3':
    con_p = line[34:43]
    con_q = line[43:53]
    con_d = line[112:113]
    #ask_p = '000000000'
    #ask_q = '000000000000'
    #bid_p = '000000000'
    #bid_q = '000000000000'
    
  elif data_type == 'B6':
    con_p = '000000000'
    con_q = '0000000000'
    con_d = '0'
    ask_p = line[34:43]
    ask_q = line[52:64]
    bid_p = line[43:52]
    bid_q = line[64:76]

  if pre_ask_p == int(ask_p) :
    move_to = '0' 
  elif pre_ask_p > int(ask_p) :
    move_to = '1'
  elif pre_ask_p < int(ask_p) :
    move_to = '2'

  pre_ask_p = int(ask_p)
  
  trimed_line = data_type + ' ' + time + ' ' + con_p + ' ' + con_q + ' ' + con_d + ' ' + ask_p + ' ' + ask_q + ' ' + bid_p + ' ' + bid_q + ' ' + move_to
  trimed_data = trimed_data + trimed_line + "\n"

df = pd.read_csv(StringIO(trimed_data), sep = " ", error_bad_lines=False)
fhand.close()

df

b'Skipping line 19: expected 10 fields, saw 11\n'


,Type,Time,C_price,C_quantity,C_direction,Ask_price,Ask_quantity,Bid_price,Bid_quantity,Move_to*
0,B6,90000254,0,0,0,30350,678,30300,1725,2
1,B6,90002010,0,0,0,30250,164,30200,199,1
2,B6,90003153,0,0,0,30200,20,30150,905,1
3,B6,90004199,0,0,0,30200,716,30150,742,0
4,B6,90005168,0,0,0,30150,81,30100,1040,1
...,...,...,...,...,...,...,...,...,...,...
33329,B6,151957740,0,0,0,29800,4726,29750,9993,0
33330,B6,151958420,0,0,0,29800,4726,29750,9993,0
33331,A3,151958487,29800,1,2,29800,4726,29750,9993,0
33332,A3,151958563,29800,152,2,29800,4726,29750,9993,0


In [5]:

cum_CQ = int()
mean_AP = df['Ask_price'].mean()
df['Pre_move*'] = 0
df['Impact*'] = 0


for i in df.index:
  if df.loc[i,'Move_to*'] == 2 :
    for k in range(30) :
      if df.loc[max(0,i-k-1), 'Move_to*'] != 0 :
        break
      df.loc[max(0,i-k-1), 'Pre_move*'] = 2
      
  if df.loc[i, 'C_direction'] == 2 :
    cum_CQ = cum_CQ + df.loc[i, 'C_quantity']
  if df.loc[i, 'Move_to*'] != 0 :
    cum_CQ = 0
  df.loc[i, 'Impact*'] = cum_CQ 
# df.loc[i, 'Impact*'] = cum_CQ / df.loc[i,'Ask_quantity'] * 100
  
df['BA_ratio*'] = df['Ask_quantity'] / df['Bid_quantity'] * 100
#df['Ask_quantity*'] = df['Ask_quantity'] / mean_AP * 100
#df['C_quantity*'] = df['C_quantity'] / mean_AP * 100

df.to_csv('df.csv')
df2 = df[['Pre_move*','Impact*','BA_ratio*']]
df2.to_csv('df2.csv')


In [6]:
df2

,Pre_move*,Impact*,BA_ratio*
0,0,0,39.304348
1,0,0,82.412060
2,0,0,2.209945
3,0,0,96.495957
4,0,0,7.788462
...,...,...,...
33329,0,627,47.293105
33330,0,627,47.293105
33331,0,628,47.293105
33332,0,780,47.293105


In [ ]:
start_time = input('Start time (HHMMSSmmm) : ')
end_time = input('End time (HHMMSSmmm) :')


after_stime = df['Time'] > int(start_time)
before_etime = df['Time'] < int(end_time)
is_move = df['Move_to*'] == 2
is_pre_move = df['Pre_move*'] == 2
fil_con = after_stime & before_etime & (is_move | is_pre_move)

df[fil_con]
   

  

  

  

Start time (HHMMSSmmm) : 100000000
End time (HHMMSSmmm) :103000000


,Type,Time,C_price,C_quantity,C_direction,Ask_price,Ask_quantity,Bid_price,Bid_quantity,Move_to*,Pre_move*,Impact*,BA_ratio*
6915,A3,101234438,30050,100,2,30050,15439,30000,23131,0,2,46.628668,0.667459
6916,A3,101234438,30050,500,2,30050,15439,30000,23131,0,2,49.867219,0.667459
6917,A3,101234439,30050,200,2,30050,15439,30000,23131,0,2,51.162640,0.667459
6918,A3,101234440,30050,500,2,30050,15439,30000,23131,0,2,54.401192,0.667459
6919,A3,101234442,30050,500,2,30050,15439,30000,23131,0,2,57.639744,0.667459
6920,A3,101234504,30050,100,2,30050,15439,30000,23131,0,2,58.287454,0.667459
6921,A3,101234527,30050,100,2,30050,15439,30000,23131,0,2,58.935164,0.667459
6922,A3,101234534,30050,500,2,30050,15439,30000,23131,0,2,62.173716,0.667459
6923,A3,101234534,30050,200,2,30050,15439,30000,23131,0,2,63.469137,0.667459
6924,A3,101234536,30050,500,2,30050,15439,30000,23131,0,2,66.707688,0.667459
